In [21]:
import pandas as pd
import numpy as np

In [22]:
train = np.load('train_preprocessed.npy', allow_pickle=True )

In [23]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

In [24]:
class Sampling(layers.Layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[1]
    dim = tf.shape(z_mean)[2]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [25]:
class anomal_dacon_encoder(tf.keras.Model) :
    def __init__(self, input_length, z_dimension):
        super(anomal_dacon_encoder, self).__init__()
        self.lstm_proj = tf.keras.layers.LSTM(input_length)
        self.dense_mean = tf.keras.layers.Dense(z_dimension)
        self.dense_log_var = tf.keras.layers.Dense(z_dimension)
        self.sampling = Sampling()

    def call(self, inputs):
        inputs = tf.keras.layers.Input(shape=(63,7))
        x = self.lstm_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        
        return z_mean, z_log_var, z


In [26]:
class anomal_dacon_generator(tf.keras.Model) :
    def __init__(self, input_length, timesteps):
        super(anomal_dacon_generator, self).__init__()
        self.lstm_proj = tf.keras.layers.LSTM(input_length , return_sequences= True)
        self.repeatvector = tf.keras.layers.RepeatVector(timesteps)

    def call(self, inputs):
        x = self.repeatvector(inputs)
        x= self.lstm_proj(x)
        
        return x 
        

In [27]:
class anomal_dacon_discriminator(tf.keras.Model) :
    def __init__(self, input_length):
        super(anomal_dacon_discriminator, self).__init__()
        self.lstm_proj = tf.keras.layers.LSTM(input_length , return_sequences= True)
        self.flatten = tf.keras.layers.Flatten()
        self.dense_toone = tf.keras.layers.Dense(1, activation='sigmoid')
    
    def call(self, inputs):
        proj_x =self.lstm_proj(inputs)
        x = self.flatten(proj_x)
        result = self.dense_toone(x)
        return proj_x, result 

In [28]:
given =  np.array([i['given'] for i in train ])

In [29]:
given.shape

(550784, 7, 63)

In [30]:
timesteps = given.shape[1]
input_length = given.shape[-1]

In [31]:
generator = anomal_dacon_generator(input_length= input_length ,timesteps = timesteps   )
discriminator = anomal_dacon_discriminator(input_length = input_length)
encoder = anomal_dacon_encoder(input_length = input_length , z_dimension = 15)

In [32]:
def discriminator_loss(x_hat, x_gen , x_real):
    dis_xhat = discriminator(x_hat)
    dis_xgen = discriminator(x_gen)
    dis_xreal = discriminator(x_real)

    return -tf.math.log(1-dis_xgen) -tf.math.log(1-dis_xhat) -tf.math.log(dis_xreal)

def generator_loss(x_hat , x_gen,  reconstruction_error):
    dis_xhat = discriminator(x_hat)
    dis_xgen = discriminator(x_gen)
    temp = -tf.math.log(dis_xgen) - tf.math.log(dis_xhat) 
    return temp + reconstruction_error

def encoder_loss(x_real, x_encoded):
    KLdivergence_loss = tf.keras.losses.KLDivergence()
    encode_loss = KLdivergence_loss(x_real, x_encoded)
    reconstruction_error = tf.keras.losses.BinaryCrossentropy(from_logits=True)(x_real, x_encoded)
    encoder_loss += reconstruction_error
    return encode_loss , reconstruction_error

In [33]:
dataset = tf.data.Dataset.from_tensor_slices(given)

In [34]:
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size=256)

In [35]:
encoder_optimizer = tf.optimizers.Adam(1e-4)
generator_optimizer = tf.optimizers.Adam(1e-4)
discriminator_optimizer = tf.optimizers.Adam(1e-4)

In [36]:
@tf.function
def train_step(x_batch_train, encoder_optimizer, generator_optimizer,discriminator_optimizer  ):
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape ,  tf.GradientTape() as enc_tape :
            log_var, mean, z = encoder(x_batch_train)
            fake_window = generator(z)
            fake_result = discriminator(fake_window)
            real_result = discriminator(x_batch_train)
            e_loss , r_loss = encoder_loss(x_batch_train. fake_window) 
            g_loss = generator_loss( z, fake_window, r_loss)
            d_loss = discriminator_loss(real_result, fake_result)
            
            gradients_of_encoder = enc_tape.gradient(e_loss , encoder.trainable_variables )
            gradients_of_generator = gen_tape.gradient(g_loss, generator.trainable_variables)
            gradients_of_discriminator = disc_tape.gradient(d_loss , discriminator.trainable_variables)
            
            encoder_optimizer.apply_gradients(zip(gradients_of_encoder, encoder.trainable_variables))
            generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

            return e_loss , g_loss , d_loss

In [37]:
e_mean = tf.metrics.Mean()
d_mean = tf.metrics.Mean()
g_mean = tf.metrics.Mean()

In [38]:
from tqdm import tqdm
for epoch in tqdm(range(100)):
    for timewindow in dataset:
        e_loss ,d_loss, g_loss = train_step(timewindow, encoder_optimizer, generator_optimizer,discriminator_optimizer)
        e_mean.update_state(e_loss)
        d_mean.update_state(d_loss)
        g_mean.update_state(g_loss)

    print('epoch: {}, e_loss: {} ,d_loss: {}, g_loss: {}'.format(epoch+1,e_mean.result() ,d_mean.result(), g_mean.result()))
    e_mean.reset_states()
    d_mean.reset_states()
    g_mean.reset_states()

0%|          | 0/100 [00:00<?, ?it/s]ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-38-6ab043591339>", line 4, in <module>
    e_loss ,d_loss, g_loss = train_step(timewindow, encoder_optimizer, generator_optimizer,discriminator_optimizer)
  File "/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py", line 580, in __call__
    result = self._call(*args, **kwds)
  File "/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py", line 627, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py", line 506, in _initialize
    *args, **kwds))
  File "/o

KeyboardInterrupt: 